In [ ]:
import random

# Directions for moving the blank space
DIRECTIONS = {
    (-1, 0): "Up",
    (1, 0): "Down",
    (0, -1): "Left",
    (0, 1): "Right"
}


# Helper functions for puzzle operations
def is_solvable(state, n):
    """Check if a puzzle is solvable."""
    flat_state = [tile for row in state for tile in row if tile != 0]
    inversions = sum(
        flat_state[i] > flat_state[j]
        for i in range(len(flat_state))
        for j in range(i + 1, len(flat_state))
    )
    return inversions % 2 == 0


def generate_puzzle(n, max_moves=None):
    """Generate a solvable puzzle for n x n grid."""
    if n == 3:
        # For n=3, generate a completely random solvable puzzle
        while True:
            tiles = list(range(n * n))
            random.shuffle(tiles)
            puzzle = [tiles[i:i + n] for i in range(0, len(tiles), n)]
            if is_solvable(puzzle, n):
                return puzzle
    else:
        # For n=4, 5, 6, scramble the goal state with limited random moves
        return scramble_puzzle(goal_state(n), n, max_moves)


def goal_state(n):
    """Return the goal state for the n-puzzle."""
    return [list(range(i * n, (i + 1) * n)) for i in range(n)]


def find_blank(state):
    """Find the position of the blank (zero) tile."""
    for i, row in enumerate(state):
        for j, tile in enumerate(row):
            if tile == 0:
                return i, j


def neighbors_with_actions(state):
    """Return all valid neighboring states and the actions to get there."""
    n = len(state)
    blank_i, blank_j = find_blank(state)
    possible_moves = []

    # Possible movements: up, down, left, right
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    for di, dj in directions:
        new_i, new_j = blank_i + di, blank_j + dj
        if 0 <= new_i < n and 0 <= new_j < n:
            # Make a copy of the state
            new_state = [row[:] for row in state]
            # Swap blank with adjacent tile
            new_state[blank_i][blank_j], new_state[new_i][new_j] = new_state[new_i][new_j], new_state[blank_i][blank_j]
            # Append the new state and the corresponding action
            possible_moves.append((new_state, DIRECTIONS[(di, dj)]))

    return possible_moves


def scramble_puzzle(puzzle, n, max_moves):
    """Scramble the goal state with a limited number of random valid moves."""
    blank_i, blank_j = find_blank(puzzle)

    for _ in range(max_moves):
        neighbors = neighbors_with_actions(puzzle)
        next_state, _ = random.choice(neighbors)  # Pick a random valid move
        puzzle = next_state

    return puzzle


def bfs(initial, goal):
    """Solve the sliding tile puzzle using BFS."""
    from collections import deque

    queue = deque([(initial, [], [])])  # Queue stores (state, path, actions)
    visited = set()  # To track visited states
    visited.add(tuple(tuple(row) for row in initial))

    max_states = 1  # Max number of states stored in memory at any point

    while queue:
        current_state, path, actions = queue.popleft()

        if current_state == goal:
            return actions, len(path), max_states

        # Explore neighbors
        for neighbor, action in neighbors_with_actions(current_state):
            neighbor_tuple = tuple(tuple(row) for row in neighbor)
            if neighbor_tuple not in visited:
                visited.add(neighbor_tuple)
                queue.append((neighbor, path + [neighbor], actions + [action]))
                max_states = max(max_states, len(queue))

    return None, 0, max_states  # In case no solution is found


# Main code to run the solver for n = 3, 4, 5, 6
def run_puzzle_solver_bfs(n_values=[3, 4, 5, 6], repetitions=10, max_moves=10):
    for n in n_values:
        print(f"\nSolving {n ** 2 - 1}-puzzle (n = {n}):")
        solution_depths = []
        max_memory_usages = []

        for i in range(repetitions):
            initial = generate_puzzle(n, max_moves=max_moves if n > 3 else None)
            goal = goal_state(n)

            # Print initial and goal state
            print(f"\nRun {i + 1}:")
            print("Initial State:")
            for row in initial:
                print(row)
            print("Goal State:")
            for row in goal:
                print(row)

            # Solve the puzzle
            actions, solution_depth, max_states = bfs(initial, goal)

            if actions is not None:
                print(f"Solution depth: {solution_depth}")
                print(f"Solution found in: {solution_depth} moves.")
                print(f"Max memory usage (states stored): {max_states}")
                print(f"Solution sequence of actions: {', '.join(actions)}")
            else:
                print("No solution found.")

            # Collect statistics
            solution_depths.append(solution_depth)
            max_memory_usages.append(max_states)

        # Report statistics
        print(f"\nStatistics for {n}-puzzle:")
        print(
            f"Solution depth: Min = {min(solution_depths)}, Max = {max(solution_depths)}, Avg = {sum(solution_depths) / len(solution_depths)}")
        print(
            f"states stored: Min = {min(max_memory_usages)}, Max = {max(max_memory_usages)}, Avg = {sum(max_memory_usages) / len(max_memory_usages)}")


# Run the solver for puzzles of size n = 3, 4, 5, 6 with 10 repetitions and puzzles solvable within max_moves
run_puzzle_solver_bfs(max_moves=25)
